In [91]:
# custom loss
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from transformers import AdamW


def my_loss(output, target):
    loss = torch.mean((output - target)**2)
    return loss

def my_cross_entropy(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    return loss


In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from transformers import AdamW

LEARNING_RATE = 0.2
NUM_EPOCHS = 100
BATCH_SIZE = 64

# data
x = torch.randn(1,3,224,224 )
# x = torch.randn(1,2)
X = torch.randn(0)
for _ in range(BATCH_SIZE*4):
    X = torch.concat((X,x),dim=0)
    
y = torch.randn(1, 77)
# y = torch.randn(1, 2)
Y = torch.randn(0)
for _ in range(BATCH_SIZE*4):
    Y = torch.concat((Y,y),dim=0)

print(X.shape,Y.shape)
# X = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
# Y = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)**2
dataset = TensorDataset(X, Y)

My_DataLoader = DataLoader(dataset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False
                        )

torch.Size([256, 3, 224, 224]) torch.Size([256, 77])


In [103]:
model = nn.Sequential(
    nn.Linear(3*224*224,2),
    nn.Tanh(),
)

model2 = nn.Sequential(
    nn.Linear(77,2),
    nn.Tanh()
)

# optimizer
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)
optimizer2 = optim.SGD(model2.parameters(),lr=LEARNING_RATE)
# optimizer = AdamW(model.parameters(), lr=1e-1)

# loss
criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
criterion = my_loss
# criterion = my_cross_entropy


model.train()
for epoch in range(NUM_EPOCHS):
    for x,y in My_DataLoader:
        optimizer.zero_grad()
        x = x.view(BATCH_SIZE,-1)
        # y = y.view(BATCH_SIZE,-1)
        
        y_pred = model(x)
        z_pred = model2(y)
        loss = criterion(z_pred, y_pred)

        loss.backward()
        optimizer.step()
        
# print(model.weight.grad)

# print(model.weight)

print('prediction')
print(y_pred[0])
print()
print('true value')
print(z_pred[0])

prediction
tensor([-1.,  1.], grad_fn=<SelectBackward0>)

true value
tensor([-0.1126,  0.0125], grad_fn=<SelectBackward0>)


In [105]:
# loss값이 얼마나 차이나는지 확인.

def my_cross_entropy(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    return loss

criterion = nn.CrossEntropyLoss()

batch_size = 5
nb_classes = 10
x = torch.randn(batch_size, nb_classes, requires_grad=True)
y = torch.randint(0, nb_classes, (batch_size,))

loss_reference = criterion(x, y)
loss = my_cross_entropy(x, y)

print(loss_reference - loss)

tensor(0., grad_fn=<SubBackward0>)
